In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json, lxml
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
from bs4 import BeautifulSoup
import warnings
import folium

Get data from Wiki

In [ ]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
soup = BeautifulSoup(requests.get(url).text)

table = soup.find('div', class_='mw-parser-output').table.tbody

cols = ['PostalCode', 'Borough', 'Neighbourhood']
data = dict({key:[]*len(cols) for key in cols})

for row in table.find_all('tr'):
    for i,column in zip(row.find_all('td'),cols):
        i = i.text
        i = i.replace('\n', '')
        data[column].append(i)

df = pd.DataFrame.from_dict(data=data)[cols]

Clean Data

In [ ]:
df = df[df.Borough != 'Not assigned'] #Only process the cells that have an assigned borough. Ignore cells with a borough that is Not assigned.

df2 = df[df.Neighbourhood != "Not assigned"]
df1 = df[df.Neighbourhood == "Not assigned"]
df1['Neighbourhood'] = df1['Borough'] #If a cell has a borough but a Not assigned  neighborhood, then the neighborhood will be the same as the borough.
#More than one neighborhood can exist in one postal code area. 
df = df1.append(df2)
print(len(df))
i = 0
postcodes = []
for index, row in df.iterrows():
  if i == 0:
    df_sel = pd.DataFrame(columns=cols)
    df_sel = df_sel.append({'PostalCode': row[0], 'Borough': row[1], 'Neighbourhood': row[2]}, ignore_index=True)
    postcodes.append(row[0])
  else:
    if row[0] in postcodes:
      existing_n = df_sel.loc[df['PostalCode'] == row[0], 'Neighbourhood']
      new_n = existing_n+","+ row[2]
      df_sel = df_sel.append({'PostalCode': row[0], 'Borough': row[1], 'Neighbourhood': new_n}, ignore_index=True)
    else:
      df_sel = df_sel.append({'PostalCode': row[0], 'Borough': row[1], 'Neighbourhood': row[2]}, ignore_index=True)
  i = i+1

df = df_sel
print(df.head())
print(df.shape)

103
  PostalCode           Borough                                Neighbourhood
0        M3A        North York                                    Parkwoods
1        M4A        North York                             Victoria Village
2        M5A  Downtown Toronto                    Regent Park, Harbourfront
3        M6A        North York             Lawrence Manor, Lawrence Heights
4        M7A  Downtown Toronto  Queen's Park, Ontario Provincial Government
(103, 3)
